# Gas Network Problem

There is a network of nodes where some gas is either produced or consumed. The goal of the problem is to find the best way to transport the gas from the production nodes to the consumption nodes.

You have to decide whether a pipeline is constructed between two nodes or not. A pipeline has a capacity and a cost to be opened (i.e, constructed). Once the pipeline is opened, you also have a cost for each unit of gas that is transported through this pipeline. Finally, there is a penalty for each unit of produced gas which is not consumed.

#### You want to minimize the total cost.

In [107]:


# consumption (demand)
d = [0, 50, 95, 10, 73, 55, 125, 32, 40, 20]
# production (maximum generation)
#gmax = [500, 0, 0, 500, 0, 0, 500, 0, 0, 0]
gmax = [500, 0, 0, 0, 0, 0, 0, 0, 0, 0]
N = len(d)
NODES = range(N)
print('generation - demand? (%d): %d' % (N, sum(gmax) - sum(d)))

generation - demand? (10): 0


In [108]:
# capacity of the arcs
ca = [ [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
       [20, 30, 40, 50, 60, 70, 80, 90, 100, 10],
       [30, 40, 50, 60, 70, 80, 90, 100, 10, 20],
       [40, 50, 60, 70, 80, 90, 100, 10, 20, 30],
       [50, 60, 70, 80, 90, 100, 10, 20, 30, 40],
       [60, 70, 80, 90, 100, 10, 20, 30, 40, 50],
       [70, 80, 90, 100, 10, 20, 30, 40, 50, 60],
       [80, 90, 100, 10, 20, 30, 40, 50, 60, 70],
       [90, 100, 10, 20, 30, 40, 50, 60, 70, 80],
       [100, 10, 20, 30, 40, 50, 60, 70, 80, 90]
     ]

# linear variable cost: the cost of transporting one unit of gas
vc = 1

# unsatisfied demand: penalty for each unit of gas which is not consumed or produced
p = 100

# linear fixed cost: the cost of opening an arc is proportional to its capacity
fc = [ [10 * c for c in row] for row in ca]


### Exercices
#### 1. Find the formulation of the problem

#### 2. Implement the model

In [109]:
import gurobipy as gp
mdl = gp.Model(name='gas network')

# Decision variables
flow_through_pipes = mdl.addVars(((i,j) for i in NODES for j in NODES if j != i), vtype=gp.GRB.CONTINUOUS, name='open', lb=0)
for i in NODES:
    flow_through_pipes[i,i] = 0

# Gas generation
gas_generation = mdl.addVars(NODES, vtype=gp.GRB.CONTINUOUS, name='generation', lb=0)

# Dependent variables

# Demand unsatisfaction
unsatisfied_demand = mdl.addVars(NODES, vtype=gp.GRB.CONTINUOUS, name='unsatisfied_demand', lb=0)

# Pipe is open indicator variable
pipe_is_open = mdl.addVars(((i,j) for i in NODES for j in NODES if j > i), vtype=gp.GRB.BINARY, name='pipe_is_open')
for i in NODES:
    pipe_is_open[i,i] = 0
for i in NODES:
    for j in NODES:
        if j < i:
            pipe_is_open[i,j] = pipe_is_open[j, i]


# Constraints
# Pipe capacity constraints
mdl.addConstrs(
       (flow_through_pipes[i, j] <= ca[i][j] * pipe_is_open[i, j] for i in NODES for j in NODES),
       name='capacity_constraints',
)

# Flow symmetry constraints
# mdl.addConstrs(
#        (flow_through_pipes[i, j] == flow_through_pipes[j, i] for i in NODES for j in NODES),
#        name='flow_symmetry_constraint',
# )

# Gas generation constraints
mdl.addConstrs(
         (gas_generation[i] <= gmax[i] for i in NODES),
         name='generation_constraints',
)

# Demand satisfaction constraints

mdl.addConstrs(
            (
                gp.quicksum(flow_through_pipes[j, i] for j in NODES) + gas_generation[i]
                == gp.quicksum(flow_through_pipes[i, j] for j in NODES) + d[i] - unsatisfied_demand[i]
                for i in NODES
            ),
            name='demand_constraints',
)

## Objective function
# Flow cost
flow_cost = gp.quicksum(vc * flow_through_pipes[i, j] for i in NODES for j in NODES)

# Unsatisfied demand cost
unsatisfied_demand_cost = gp.quicksum(p * unsatisfied_demand[i] for i in NODES)

# Opening a pipe cost
opening_pipe_cost = gp.quicksum(fc[i][j] * pipe_is_open[i, j] for i in NODES for j in NODES if i > j)

mdl.setObjective(flow_cost + unsatisfied_demand_cost + opening_pipe_cost, sense=gp.GRB.MINIMIZE)


In [110]:
mdl.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.5.0 23F79)
Gurobi Compute Server Worker version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8275CL CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 120 rows, 155 columns and 390 nonzeros
Model fingerprint: 0xa8bd8e69
Variable types: 110 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 5e+02]
Found heuristic solution: objective 50000.000000
Presolve removed 20 rows and 9 columns
Presolve time: 0.00s
Presolved: 100 rows, 146 columns, 371 nonzeros
Variable types: 101 continuous, 45 integer (45 binary)

Root relaxation: objective 7.403000e+03, 65 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |   

In [111]:
print("Sanity check")
print("  Total generation:", sum(gas_generation[i].x for i in NODES))
print("  Total demand:", sum(d[i] for i in NODES))
print("  Total deficit:", sum(unsatisfied_demand[i].x for i in NODES))

Sanity check
  Total generation: 500.0
  Total demand: 500
  Total deficit: 0.0


In [112]:
mdl.write("gas_network.lp")
if mdl.status == gp.GRB.OPTIMAL:
    print("Optimal solution found")
    for i in NODES:
        print(f"Node {i}:")
        print(f"  Generation: {gas_generation[i].x}")
        print(f"  Deficit: {unsatisfied_demand[i].x}")
        print(f"  Demand: {d[i]}")

        node_flows = {
            (i, j): expression.getValue()
            if isinstance((expression := flow_through_pipes[i, j]), gp.LinExpr)
            else (
                expression.X if isinstance(expression, gp.Var)
                else expression
            )
            for i in NODES for j in NODES
        }
        stringified = {j: f"{max(node_flows[i, j],-node_flows[j, i], key=abs):02.2f}" for j in NODES}
        print(f"  Mass flow to nodes {stringified}")
        print(f"  Sums to {sum(node_flows[i, j] for j in NODES)}")

else:
    print("No optimal solution found")
    mdl.computeIIS()
    mdl.write("gas_network.ilp")

Optimal solution found
Node 0:
  Generation: 500.0
  Deficit: 0.0
  Demand: 0
  Mass flow to nodes {0: '0.00', 1: '20.00', 2: '30.00', 3: '40.00', 4: '50.00', 5: '55.00', 6: '70.00', 7: '80.00', 8: '63.00', 9: '92.00'}
  Sums to 500.0
Node 1:
  Generation: 0.0
  Deficit: 0.0
  Demand: 50
  Mass flow to nodes {0: '-20.00', 1: '0.00', 2: '0.00', 3: '-30.00', 4: '0.00', 5: '0.00', 6: '0.00', 7: '0.00', 8: '0.00', 9: '0.00'}
  Sums to 0.0
Node 2:
  Generation: 0.0
  Deficit: 0.0
  Demand: 95
  Mass flow to nodes {0: '-30.00', 1: '0.00', 2: '0.00', 3: '0.00', 4: '0.00', 5: '0.00', 6: '0.00', 7: '-48.00', 8: '0.00', 9: '-17.00'}
  Sums to 0.0
Node 3:
  Generation: 0.0
  Deficit: 0.0
  Demand: 10
  Mass flow to nodes {0: '-40.00', 1: '30.00', 2: '0.00', 3: '0.00', 4: '0.00', 5: '0.00', 6: '0.00', 7: '0.00', 8: '0.00', 9: '0.00'}
  Sums to 30.0
Node 4:
  Generation: 0.0
  Deficit: 0.0
  Demand: 73
  Mass flow to nodes {0: '-50.00', 1: '0.00', 2: '0.00', 3: '0.00', 4: '0.00', 5: '0.00', 6: '0.0